In [1]:
import pandas as pd

In [2]:
from utils.lipht_visualization import topic_distribution_barplot
from utils.lipht_lda_utils import PrepareDictionary, lda_predict_df, df_lda_features, get_topics_and_probability, get_lda_topics, df_lda_preprocessing, TrainLDAModel
from utils.lipht_data import getEngine

In [52]:
engine = getEngine('LI-PH-01','Akademikernes_MSCRM')#,'script','Google100123!', 'MSSQLSERVER')

In [4]:
df_messages = pd.read_pickle('data/LDA_Messages_persisted_with_language.pkl')

In [54]:
df_messages.head(2)

,ThreadID,ThreadSubject,FirstMessage,FirstMemberMessage,ThreadInitiatedBy,ThreadClass,InDiagnosticScope,ThreadMessageID,ThreadResponsibleDepartment,ThreadResponsibleDepartmentTeam,...,text_Exclamationmarks,tokenized_text,stopwords_removed,lemmatized_text,stemmed_text,bow,language_prediction,language_probability,language_id,language
0,15FA2A2B-B6FA-E611-AC9F-005056AD2D14,Nyt brev: Is part time insurance right for you?,"Hej,\n\n\nWell, yes I did. Basically because I...","Hej,\n\n\nWell, yes I did. Basically because I...",Member,Member to AKA to Member,1,1EFA2A2B-B6FA-E611-AC9F-005056AD2D14,Forsikring,Medlemskabsteam,...,0,"[have_a, request, job, i_say, have, one_month,...","[have_a, request, job, i_say, one_month, gradu...","[have_a, request, job, i_say, one_month, gradu...","[have_a, request, job, i_say, one_month, gradu...","[(68, 1), (78, 1), (109, 1), (110, 1), (113, 1...","[0.964599, 1, English]",0.964599,1,English
1,36B7B25E-A349-E711-BE8E-005056AD2D14,Virksomhedspraktik samt samtale med A-kasse og...,"Hej,\nSom aftalt med Karen Henningsen ved møde...","Hej,\nSom aftalt med Karen Henningsen ved møde...",Member,Member to AKA to Member,1,3DB7B25E-A349-E711-BE8E-005056AD2D14,Jobmatch,Team Rådighed,...,0,"[som_aftalt, nu_til, kalde, vide_at, meddeler,...","[som_aftalt, nu_til, kalde, vide_at, meddeler,...","[som_aftalt, nu_til, kalde, vide_at, meddeler,...","[som_aftalt, nu_til, kald, vide_at, meddel, vi...","[(27, 1), (126, 1), (180, 1), (185, 1), (218, ...","[0.96779394, 0, Danish]",0.967794,0,Danish


In [5]:
# Overview of teams and row count for ThreadMessageID
# df_messages[['ThreadResponsibleDepartmentTeam', 'ThreadMessageID']].groupby(['ThreadResponsibleDepartmentTeam']).agg(['count'])

In [6]:
respteams = ['Job','Logistik','Medlemskabsteam','Medlemskabsteam', 'Udbetalingsteam']

In [7]:
departmentteam = 'Udbetalingsteam'
initiatedby = 'Member'
n_gram = 3
sample_size= 10000
no_above = 0.014
no_below= 2 # filter out tokens that appear in less than 15 documents
random_state=1
lda_num_topics = 50
lda_chunksize = 1000
lda_passes = 50

log = {
    'departmentteam': departmentteam,
    'initiatedby': initiatedby,
    'n_gram': n_gram,
    'no_above': no_above,
    'no_below': no_below,
    'random_state': random_state,
    'lda_num_topics': lda_num_topics,
    'lda_chunksize': lda_chunksize,
    'lda_passes': lda_passes
}

In [8]:
df_team = df_messages[(df_messages['FirstMessage'].isnull()==False) & (df_messages['ThreadResponsibleDepartmentTeam'].str.contains(departmentteam)==True) & (df_messages['ThreadInitiatedBy']==initiatedby) & (df_messages['language']=='Danish')].copy(deep=True)
log['rows'] = df_team.shape[0]

In [9]:
df_lda_preprocessing(df_team, 'text')

C:\tools\Anaconda3\envs\aka-diagnostic\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [10]:
# Create dictionary and corups
dictionary, corpus = PrepareDictionary(df_team, 'ngrams', no_above, no_below, log)

C:\tools\Anaconda3\envs\aka-diagnostic\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Found 280331 words.
Left with 42456 words.
bow column added to df


In [11]:
# Create identifier
log['identifier'] = '{0}-Initiatedby_{1}_Topics-{2}'.format(departmentteam, initiatedby, lda_num_topics)

In [12]:
%%time
# Train the model
team_lda_model = TrainLDAModel(corpus, dictionary, lda_num_topics, 4, lda_chunksize, lda_passes, random_state, log, directory=log['identifier'])
# team_lda_model.save('{0}_model'.format(log['identifier']))
# dictionary.save('{0}_dictionary.pkl'.format(log['identifier']))

Wall time: 1min 19s


In [13]:
# Get the distribution
title = '{0}_Distribution'.format(log['identifier'])
topic_distribution_barplot(team_lda_model, df_team, 5, title)

In [15]:
# Get topics and probability
topics = get_topics_and_probability(df_team, team_lda_model, lda_num_topics, 40)
topics['team'] = pd.Series(log['departmentteam'], index=topics.index)
topics['initiatedby'] = pd.Series(log['initiatedby'], index=topics.index)
topics['lda_num_topics'] = pd.Series(log['lda_num_topics'], index=topics.index)

In [53]:
topics.to_sql(name='topics_index' ,con=engine , schema='input', if_exists='append', index=False)

In [ ]:
# Create dataset for predictions
team_prediction = df_team[['ThreadID','ThreadMessageID','ThreadInitiatedBy','text','stemmed_text','ngrams','bow']].copy(deep=True)
team_prediction['lda_num_topics'] = pd.Series(log['lda_num_topics'], index=team_prediction.index)
team_prediction

In [ ]:
lda_predict_df(team_prediction,'ngrams', team_lda_model, dictionary)

In [ ]:
team_prediction = team_prediction[['ThreadID','ThreadMessageID','ThreadInitiatedBy','lda_num_topics','pred_probability','pred_index']]
team_prediction

In [ ]:
team_prediction.to_sql(name='topics_predictions',con=engine , schema='input', if_exists='append', index=False)

In [ ]:
df_log = pd.DataFrame(log, index=['0'])
df_log

In [ ]:
df_log.to_sql(name='log_multi_model' ,con=engine , schema='input', if_exists='append', index=False)